# Setting Up a Local Database in TinyDB for Fast RAG on Personal Questions
In testing and receiving feedback from some of my initial users, one thing I found was that most of my users expected the AI to be able to answer basic questions about me in a way that it wasn't. To solve this problem, I'm going to create a local RAG system of questions and answers to common interview questions. This database is built with [TinyDB](https://tinydb.readthedocs.io/en/latest/index.html), which offers efficient and lightweight algorithms for keeping a small, local DB. I will continue adding my answers to common questions to this dataset to improve the quality of answers.

In [1]:
from tinydb import TinyDB, Query
from GoogleEmbeddings import Embeddings

This code splits the text read in from the file, and embeds it using the [GoogleEmbeddings](https://github.com/mocboch/website/blob/master/GoogleEmbeddings.py) class.

In [2]:
with open('personal_info.txt') as f:
    text = f.read()
text = text.split('\n* ~')
ls = [t.strip('~* ') for t in text]
dicts = []
embeddings = Embeddings(api_key=open('google_api_key.txt').read())
for i in range(len(ls)): 
    if i%2 == 0:
        dicts.append({'question': ls[i],'answer': ls[i+1], 
                      'question-embedded': embeddings.embed_query(ls[i]),
                      'answer-embedded': embeddings.embed_query(ls[i+1])})

    

Here is an example of a question and answer that I'll be inserting into the database.

In [39]:
dicts[0]['question']

'Tell me about Mark'

In [40]:
dicts[0]['answer']

'Mark grew up in New Jersey, and first got interested in code when he learned to program using DarkBASIC in middle school. He attended an IT-focused high school program where he continued to develop his skills, adding programming in Python and Java to his skillset along with certifications in SQL, Microsoft Excel, and Comptia A+. He went on to attend Alfred University for Environmental Science. After that, Mark had a couple of jobs in that field, including working as a Park Ranger, and as an intern working with critically endangered birds. Following that, he discovered a passion for cooking and pursued it for years, eventually becoming a kitchen manager. When he decided it was time for a change, he went back to school for a Master’s degree in data science, which he is now finishing. Mark is very interested in Natural Language Processing and Bayesian Statistics.'

## Creating the DB with TinyDB
This cell creates a DB and a query object in TinyDB, and the next inserts the dictionaries created in the previous step as documents. That function returns a list of the doc_ids associated with the inserts. 

In [3]:
db = TinyDB('personal-info.json')
User = Query()

In [4]:
db.insert_multiple(dicts)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

The next two cells create a table object with the [default table](https://tinydb.readthedocs.io/en/latest/usage.html#default-table) created by TinyDB, and then a vector search object with a pair of vector and scalar indices for each entry. For now, I have a small enough dataset that I can hold all of these vectors in memory, and use a naive algorithm to find the best match, but I will revisit this in the near future to create a more efficient search algorithm that considers time and space complexity. 

In [5]:
table = db.table('_default')

In [14]:
vec_search = [tuple((t.doc_id, t['question-embedded'])) for t in table.all()]

This cell defines a function to provide cosine similarity scores. Cosine similarity scores are a commonly used metric of comparison between vector embeddings.

In [62]:
import numpy as np
def cosine_similarity(vec1: list[float], vec2: list[float]) -> float:
    num = np.dot(vec1, vec2)
    denom = np.sqrt(np.dot(vec1, vec1) * np.dot(vec2, vec2))
    return num/denom

Now, I can test it out. Below are some test queries, the resulting cosine similarity scores, and the highest scoring questions in the dataset.

In [63]:
import pandas as pd
query = 'Give me a reason I should hire Mark'
embedded_query = embeddings.embed_query(query)
scores = [tuple((row[0], cosine_similarity(embedded_query, row[1]))) for row in vec_search]
pd.DataFrame(scores).set_index(0).sort_values(ascending=False, by=1).rename(columns={0:'doc_id', 1:'score'})

,score
0,
4,0.952122
1,0.890730
7,0.884191
13,0.851925
6,0.841705
8,0.833763
3,0.822064
5,0.821628
9,0.819010


In [26]:
table.get(doc_id=4)['question']

'Why should we hire Mark?'

In [50]:
query = 'What should I know about Mark?'
embedded_query = embeddings.embed_query(query)
scores = [tuple((row[0], cosine_similarity(embedded_query, row[1]))) for row in vec_search]
pd.DataFrame(scores).set_index(0).sort_values(ascending=False, by=1).rename(columns={0:'doc_id', 1:'score'})

,score
0,
1,0.951057
8,0.932492
9,0.925750
7,0.915157
4,0.913031
3,0.898955
6,0.893020
5,0.884995
10,0.855615


In [52]:
table.get(doc_id=1)['question']

'Tell me about Mark'

In [53]:
table.get(doc_id=8)['question']

'What are Mark’s goals?'

In [31]:
table.get(doc_id=9)['question']

'What are Mark’s interests?'

In [54]:
query = 'What does Mark like to do on the weekends?'
embedded_query = embeddings.embed_query(query)
scores = [tuple((row[0], cosine_similarity(embedded_query, row[1]))) for row in vec_search]
pd.DataFrame(scores).set_index(0).sort_values(ascending=False, by=1).rename(columns={0:'doc_id', 1:'score'})

,score
0,
3,0.929328
9,0.897411
1,0.879798
5,0.871701
8,0.868609
4,0.844823
7,0.840330
13,0.838112
2,0.817903


In [55]:
table.get(doc_id=3)['question']

'What are Mark’s hobbies?'

In addition to implementing a more efficient search algorithm, and adding more questions and answers to the dataset, in the near future I would like to create amn training set of some queries paired with 'golden documents.' This will allow me to get a concrete assessment of the model's accuracy, which in turn will allow me to investigate reducing the dimensionality of the vector index further with a reliable approach. I would also like to implement a 'skills' section of this database, where I will write up paragraphs about each of my skills, pulled from job descriptions I'm looking at, and then allow the model to search using string matching TinyDB's own search algorithms. 

The final system will allow the machine to respond to users with context from: 
- My answers to interview questions, based on their similarity to the user's query
- My skills, based on information the user has provided about the job they are interested in hiring for and searched by keyword

This system can also be integrated into the personalized resume creation process by generating skill descriptions that match keywords in the job description context